In [2]:
from ipywidgets import GridspecLayout
from ipydatagrid import DataGrid, TextRenderer, VegaExpr
import ipywidgets as widgets
import ipydatagrid
import pandas as pd
from astropy.table import Table
import eb_predictor
from astropy.time import Time
from spinner_alone import Spinner
from datetime import datetime, UTC
from astropy import units as u

class Page(widgets.Box):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        style = {'description_width': 'initial'}
        
        #Text Widget
        self.text = widgets.HTML(
            value="<h1>Visible Eclipsing Binary Predictor</h1>"
                "This is a tool to predict when the primary eclipse is observable. " 
                "After the information is input into the boxes, a table will generate below after submit is pressed. "
                "Values can be changed and hitting the submit button will reload/recalculate the table."
        )

        #Widget for date picker
        self.obs_date = widgets.DatetimePicker(
            description='Start Date and Time',
            value=datetime.now(UTC).astimezone(),
            disabled=False,
            style=style
        )

        # Widget for Days in the Future
        self.days_future = widgets.BoundedIntText(
            description="Days in the future",
            value=1,
            min=1,
            max=30,
            disabled = False,
            style=style,
            tooltip="Enter number of days in the future for possible objects to observe."
        )

        #Widget for Longitude
        self.long = widgets.BoundedFloatText(
            description="Longitude",
            value=0,
            min=-180.0,
            max=+180.0,
            disabled=False,
            tooltip="Enter in decimal degrees."
        )
        
        #Widget for Latitude
        self.lat=widgets.BoundedFloatText(
            description="Latitude",
            value=0,
            min=-90.0,
            max=+90.0,
            disabled=False,
            tooltip="Enter in decimal degrees, N is positive, S is negative."
        )

        #Widget for Elevation
        self.elev = widgets.FloatText(
            description='Elevation (m)',
            disabled=False,
            tooltip="Enter in decimal meters from sea level."
        )

        #Widget for Altitude
        self.alti = widgets.BoundedFloatText(
            description='Minimum Altitude',
            min=0,
            max=90,
            value=30,
            disabled=False,
            tooltip="Enter in decimal degrees."
        )

        #Widget for Spinner
        self.spinner = Spinner(spinner_file="star_spinner.svg")

        #Widget for Table/Spinner Placeholder
        self.table = widgets.VBox([self.spinner], layout={"height": "40em"})

        #Widget for Submit Button: Includes table formatting
        submit_button = widgets.Button(description='Submit')
        def on_submit(button):
            
            self.table.children = [self.spinner]
            self.spinner.start()
            self.ebp = eb_predictor.EclipsingBinaryPredictor(
                start_time_str = self.obs_date.value, 
                days_in_future=self.days_future.value,
                minimum_altitude = self.alti.value,
                latitude = self.lat.value,
                longitude = self.long.value,
                elevation = self.elev.value,
            )
            
            self.ebp.init()
            no_eclipses=False
            try:
                self.ebp.main()
            except ValueError:
                no_eclipses=True

            if no_eclipses:
                self.text2 = widgets.HTML(
                    value="No eclipses found."
                )
                self.table.children = [self.text2]
                return
            
            self.ebp.predictions["Can See"] = "SOMETHING IS WRONG"
            self.ebp.predictions["Can See"][self.ebp.predictions["FirstHalf"]] = "First"
            self.ebp.predictions["Can See"][self.ebp.predictions["SecondHalf"]] = "Second"
            self.ebp.predictions["Can See"][self.ebp.predictions["FullEclipse"]] = "Full"
            self.ebp.predictions["MaxMag"]=[float(star.split()[0]) for star in self.ebp.predictions["MaxMag"]]
            self.ebp.predictions.rename_column("RA2000", "RA")
            self.ebp.predictions.rename_column("Declination2000", "Dec")
            self.ebp.predictions.rename_column("EclipseDuration","Duration (h)")
            self.ebp.predictions.rename_column("MidtransitTimes","Mid Eclipse")
            self.ebp.predictions.rename_column("EclipseStart", "Start")
            self.ebp.predictions.rename_column("EclipseEnd","End")
            self.ebp.predictions["Duration (h)"]=self.ebp.predictions["Duration (h)"].to(u.hour)
            del self.ebp.predictions["ProperMotionRA", 
                "ProperMotionDec",
                "VariabilityType",
                "OID",
                "Constellation",
                "EclipseDate",
                "Category",
                "SpectralType",
                "FirstHalf",
                "SecondHalf",
                "FullEclipse",
                "MinMag",
                "Period",
                "Epoch",
                "AUID"
            ]
            
            df = self.ebp.predictions.to_pandas()
            df["Mid Eclipse"] = [f.round(freq='s') for f in df["Mid Eclipse"]]
            df["Start"] = [f.round(freq='s') for f in df["Start"]]
            df["End"] = [f.round(freq='s') for f in df["End"]]
            date_render = TextRenderer(text_value=VegaExpr("utcFormat(toDate(cell.value), '%Y-%m-%d %H:%M:%S')"))
            duration_render = TextRenderer(format=".2f")
            renderer = {"Mid Eclipse": date_render, "Start": date_render, "End": date_render, "Duration (h)":duration_render}
            self.dg2 = DataGrid(df, renderers=renderer, selection_mode="cell")
            self.table.children = [self.dg2] 
            self.dg2.auto_fit_columns = True
        submit_button.on_click(on_submit)
        self.submit = submit_button

        # Creates Widget Grid
        self.grid = GridspecLayout(6, 5, height='400px')
        self.grid[0,:4] = self.text
        self.grid[1, 1] = self.obs_date 
        self.grid[2,1] = self.days_future
        self.grid[3, 0] = self.elev 
        self.grid[1, 0] = self.long
        self.grid[2, 0] = self.lat
        self.grid[3, 1] = self.alti
        self.grid[2, 3] = self.submit
        self.grid[5, :4] = self.table
        
        self.children = [self.grid]


# Goals:
# Single object functionality
# Mobile Functionality
# Store Observer Location

In [3]:
eb = Page()
eb


Page(children=(GridspecLayout(children=(HTML(value='<h1>Visible Eclipsing Binary Predictor</h1>This is a tool …